In [ ]:
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent,load_tools,AgentType
from dotenv import load_dotenv
import os
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
serpapi_api_key = os.getenv("SERPAPI_API_KEY")

llm=ChatGroq(temperature=0.7,model="llama3-8b-8192")

tools=load_tools(["serpapi"],llm=llm)

agent=initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
query="What is the latest news about AI in technology"
response=agent.run(query)

print("Final Answer")
print(response)

In [ ]:
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, AgentType, Tool
import requests
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
serpapi_api_key = os.getenv("SERPAPI_API_KEY")

# 1. Groq LLM
llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model_name="llama3-70b-8192"
)

# 2. Custom SerpAPI tool using requests
def event_finder(location: str) -> str:
    api_key = os.environ["SERPAPI_API_KEY"]
    query = f"events near {location}"
    
    params = {
        "engine": "google",
        "q": query,
        "api_key": api_key
    }

    try:
        response = requests.get("https://serpapi.com/search", params=params)
        data = response.json()
        results = data.get("organic_results", [])
        if not results:
            return f"[EventFinder] No events found for: {location}"
        top_results = "\n".join([f"- {r.get('title')}" for r in results[:3]])
        return f"[EventFinder] Events near {location}:\n{top_results}"
    except Exception as e:
        return f"[EventFinder] Error fetching events: {str(e)}"

# 3. Book Recommender Tool
def book_recommender(query: str) -> str:
    genre = query.lower()
    recommendations = {
        "science fiction": ["Dune", "Neuromancer"],
        "fantasy": ["The Hobbit", "Mistborn"],
        "romance": ["Pride and Prejudice", "The Notebook"],
        "mystery": ["Gone Girl", "The Girl with the Dragon Tattoo"],
    }
    for key in recommendations:
        if key in genre:
            return f"[BookRecommender] Recommended {key} books: {', '.join(recommendations[key])}"
    return f"[BookRecommender] No recommendations found for: '{query}'"

# 4. Travel Guide Tool
def travel_guide(location: str) -> str:
    guides = {
        "paris": "Visit Eiffel Tower, Louvre, and cruise the Seine.",
        "new york": "Explore Times Square, Central Park, and Statue of Liberty.",
        "tokyo": "See Shibuya Crossing, Tokyo Tower, and enjoy sushi.",
        "delhi": "See India Gate, Qutub Minar, and try Chandni Chowk street food.",
    }
    loc = location.lower()
    return guides.get(loc, f"[TravelGuide] No guide available for '{location}'")

# 5. Define Tools
tools = [
    Tool(
        name="Event Finder",
        func=event_finder,
        description="Finds local events using SerpAPI given a location"
    ),
    Tool(
        name="Book Recommender",
        func=book_recommender,
        description="Recommends books based on genre like 'fantasy', 'romance', etc."
    ),
    Tool(
        name="Travel Guide",
        func=travel_guide,
        description="Gives travel tips for major cities"
    )
]

# 6. Initialize the ReAct Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 7. Query
query = "Recommend a fantasy book, show me events in Delhi, and guide me around Paris."

# 8. Run the agent
response = agent.run(query)

# 9. Print output
print("\n🧠 Final Answer:\n", response)
